In [109]:
from core.project.project import Project
from sys import maxsize
import numpy as np
import time

p = Project()
# p.load('/Users/flipajs/Documents/wd/180713_1633_Cam1_clip_initial')
p.load('/Users/flipajs/Documents/wd/ferda/180819_Sowbug3_cut_new_setup')
# p.load('/Users/flipajs/Documents/wd/ferda/180811_0832_5Zebrafish_nocover_22min_fixed_cardinality')
# p.load('/Users/flipajs/Documents/wd/ferda/180811_0205_Camera3-5min_fixed_cardinality')

Initializing db at /Users/flipajs/Documents/wd/ferda/180819_Sowbug3_cut_new_setup/rm.sqlite3 
saving chm
saving GM


In [88]:
from utils.gt.gt import GT
import tqdm

correct_labels = {}
# gt_path = 'data/GT/Cam1_clip.avi.pkl'
# gt_path = 'data/GT/5Zebrafish_nocover_22min.pkl'
gt_path = 'data/GT/Sowbug3.pkl'
# gt_path = 'data/GT/Camera3-5min.mp4.pkl'
print(p.video_start_t)
gt = GT()
gt.load(gt_path)

cy, cx = 0, 0
frame_offset = p.video_start_t
# TODO: only for Sowbugs3
frame_offset = 806
if p.video_crop_model is not None:
    cy = p.video_crop_model['y1']
    cx = p.video_crop_model['x1']

gt.set_offset(y=cy,
              x=cx,
              frames=frame_offset)

for tracklet in tqdm.tqdm_notebook(p.chm.tracklet_gen(), total=len(p.chm)):
    id_set = gt.tracklet_id_set(tracklet, p)

    correct_labels[tracklet.id()] = len(id_set)

0


GT was sucessfully loaded from data/GT/Sowbug3.pkl


  0%|          | 0/967 [00:00<?, ?it/s]

  7%|▋         | 68/967 [00:00<00:01, 679.80it/s]

 14%|█▍        | 136/967 [00:00<00:01, 678.58it/s]

...

100%|██████████| 37/37 [00:00<00:00, 879.66it/s]


  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 826.95it/s]


  0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:00<00:00, 875.27it/s]

In [110]:
from experiments.cardinality_classifier import global_cardinality_classifier as g_cardinality_clf
reload(g_cardinality_clf)
collateral_sets = g_cardinality_clf.get_collateral_sets(p, max_frame=5000)
tracklets = set([t for cs in collateral_sets for t in cs])
print("#unique tracklets: {}, #collateral sets: {}".format(len(tracklets), len(collateral_sets)))

#unique tracklets: 138, #collateral sets: 276


In [111]:
t = time.time()
predecessors = g_cardinality_clf.generate_predecessor_map(tracklets, p)
successors = g_cardinality_clf.generate_successor_map(tracklets, p)
print("predecessors t: {}".format(time.time() - t))

num_animals = len(p.animals)
t = time.time()
areas = g_cardinality_clf.get_median_areas(tracklets, p)
# TODO: medain_area should probably be median from all areas...
median_area = np.median(areas.values())
print("area median extraction time: {}".format(time.time() - t))
area_relaxation_coef = 0

print("#animals {}, #tracklets: {}, #collateral sets: {}, mean area: {}".format(num_animals, len(tracklets), len(collateral_sets), median_area))

cardinalities = g_cardinality_clf.build_ilp_and_solve(tracklets, collateral_sets, predecessors, successors, num_animals, 
                                                      median_area, areas, 
                                                      area_relaxation_coef=area_relaxation_coef, print_ilp=False)

predecessors t: 0.0260760784149


area median extraction time: 2.14833807945
#animals 5, #tracklets: 138, #collateral sets: 276, mean area: 436.5
relaxation_coef: 0
var registration t: 0.0601828098297


objective function: 0.571121931076


constraints: 0.589616060257
ILP construction tooks: 1.22222900391s
('Status:', 'Optimal')
('Total Cost = ', 1471383.0)
time: 0.156780958176s
1


In [91]:
import copy
# ants3_tracklets = copy.copy(tracklets)
# ants3_cardinalities = copy.copy(cardinalities)

# sowbug3_tracklets = copy.copy(tracklets)
# sowbug3_cardinalities = copy.copy(cardinalities)

# zebrafish_tracklets = copy.copy(tracklets)
# zebrafish_cardinalities = copy.copy(cardinalities)
# correct_labels[7586] = 1
# correct_labels[4368] = 1
# correct_labels[4349] = 1
# correct_labels[4438] = 1

In [101]:
tracklets_len = {t.id(): len(t) for t in tracklets}

# for coef in [0, 0.1, 0.2, 0.3, 0.4, 0.5]:
for coef in [0]:
    # cardinalities = g_cardinality_clf.build_ilp_and_solve(tracklets, collateral_sets, predecessors, num_animals,
    #                                                       median_area, areas, area_relaxation_coef=coef)
    num_mistakes = 0
    
    tracklets_len_sum = 0
    
    for id in correct_labels.keys():
        if correct_labels[id] != cardinalities[id]:
            print("mistake, id: {}, correct cardinality: {}, estimated: {}".format(id, correct_labels[id], cardinalities[id]))
            num_mistakes += 1
            tracklets_len_sum += tracklets_len[id]
            
    print("COEF: {}, #mistakes: {}, len: {}".format(coef, num_mistakes, tracklets_len_sum))
    print("------------------------------\n")

COEF: 0, #mistakes: 0, len: 0
------------------------------



In [94]:
num_mistakes = 0
tracklets_len_sum = 0

for t in tracklets:
    try:
        card = correct_labels[t.id()]
        if card == 0 and t.is_noise():
            pass
        elif card == 1 and t.is_single():
            pass
        elif card > 1 and t.is_multi():
            pass
        else:
            print("mistake, id: {}, correct cardinality: {}, estimated: {}".format(t.id(), correct_labels[t.id()], t.segmentation_class_str()))
            num_mistakes += 1
            tracklets_len_sum += len(t)
    except:
        print(t.id())
    
print("#mistakes: {}, len: {}".format(num_mistakes, tracklets_len_sum))
print("------------------------------\n")

mistake, id: 4722, correct cardinality: 1, estimated: no-ID
mistake, id: 374, correct cardinality: 2, estimated: single-ID
mistake, id: 4233, correct cardinality: 1, estimated: no-ID
#mistakes: 3, len: 243
------------------------------

